# Neighborhood Computations

In [1]:
import numpy as np
import igl
import meshplot

In [2]:
bumpy_v, bumpy_f = igl.read_triangle_mesh("data/bumpy.off")
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
car_v, car_f = igl.read_triangle_mesh("data/car.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
cup_v, cup_f = igl.read_triangle_mesh("data/coffeecup.off")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

faces = cube_f
vertices = cube_v


## Vertex-to-Face Relations

In [3]:
def vertex_to_face_relations(vertices, faces) -> None:
    adjacency_list, cumulative_sum = igl.vertex_triangle_adjacency(faces, len(vertices))

    face = vertex = 0
    for idx in range(len(vertices)):
        print(f'Vertex {idx + 1} is adjacent to the faces', end=': ')
        for _ in range(cumulative_sum[vertex + 1] - cumulative_sum[vertex]):                        
            print(f'{adjacency_list[face]}' , end='  ')
            face += 1
        print()
        vertex += 1
    
vertex_to_face_relations(bunny_v, bunny_f)


Vertex 1 is adjacent to the faces: 849  850  912  944  945  
Vertex 2 is adjacent to the faces: 738  788  1278  1833  
Vertex 3 is adjacent to the faces: 248  2795  2857  2972  3408  
Vertex 4 is adjacent to the faces: 911  912  945  960  974  991  
Vertex 5 is adjacent to the faces: 971  2977  2984  3012  3013  
Vertex 6 is adjacent to the faces: 1455  1658  1706  2471  4816  4884  
Vertex 7 is adjacent to the faces: 1906  2443  2506  2518  2526  
Vertex 8 is adjacent to the faces: 5  15  103  2576  2697  2799  
Vertex 9 is adjacent to the faces: 55  118  144  2711  3005  
Vertex 10 is adjacent to the faces: 2599  2729  2805  2945  3066  
Vertex 11 is adjacent to the faces: 96  941  2736  2773  2803  2832  3164  3165  
Vertex 12 is adjacent to the faces: 13  96  1082  1689  2736  3137  3143  
Vertex 13 is adjacent to the faces: 1689  3137  3816  3911  
Vertex 14 is adjacent to the faces: 1037  1053  1054  1087  1122  1133  1134  
Vertex 15 is adjacent to the faces: 90  107  123  139  

## Vertex-to-Vertex Relations

In [4]:
def vertex_to_vertex_relations(faces) -> None:
    adjacency_list = igl.adjacency_list(faces)
    
    for idx, vertices in enumerate(adjacency_list):
        print(f'Vertex {idx} is adjacent to the following vertices: {", ".join([str(v) for v in vertices])}.') 
    
vertex_to_vertex_relations(bunny_f)

Vertex 0 is adjacent to the following vertices: 3, 24, 308, 525, 542.
Vertex 1 is adjacent to the following vertices: 415, 549, 551, 596.
Vertex 2 is adjacent to the following vertices: 134, 287, 465, 497, 1308.
Vertex 3 is adjacent to the following vertices: 0, 246, 308, 510, 525, 543.
Vertex 4 is adjacent to the following vertices: 406, 1235, 1368, 1371, 1375.
Vertex 5 is adjacent to the following vertices: 792, 917, 1113, 3426, 3454, 3482.
Vertex 6 is adjacent to the following vertices: 484, 1130, 1163, 1179, 1191.
Vertex 7 is adjacent to the following vertices: 1236, 1267, 1352, 1381, 1382, 2146.
Vertex 8 is adjacent to the following vertices: 74, 1325, 1365, 1402, 1420.
Vertex 9 is adjacent to the following vertices: 280, 465, 1225, 1227, 1259.
Vertex 10 is adjacent to the following vertices: 11, 349, 451, 1319, 1524, 1537, 1565, 1671.
Vertex 11 is adjacent to the following vertices: 10, 12, 349, 1413, 1524, 1542, 1693.
Vertex 12 is adjacent to the following vertices: 11, 1542, 16

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

In [5]:
def explode(vertices, faces) -> tuple[list, list]:
    exploded_faces = np.zeros(faces.shape, dtype=int)
    # Needs to have enough space for unique vertices so each face * 3.     
    exploded_vertices = np.zeros((len(faces) * 3, 3))
    vertex_idx = 0
    
    for idx, face in enumerate(faces):
        exploded_faces[idx] = [vertex_idx + x for x in range(3)]
        for x in range(3):
            exploded_vertices[vertex_idx + x] = vertices[face[x]]
        
        vertex_idx += 3
    return exploded_vertices, exploded_faces

def plotShader(v, f, n, threshold=0.01, flat=False, wireframe=False) -> None:
    p = meshplot.plot(v, f, n=n, shading={'flat': flat, 'wireframe' : wireframe})
    # p.add_lines(v, v + threshold*n)
    

### Flat Shading

In [6]:
def flat_shading(vertices, faces) -> None:
    # Get degenerate normal
    degen_norm = np.array([1, 1, 1]) / np.sqrt(np.sum(vertices**2))
    v, f = explode(vertices, faces)
    normals = igl.per_face_normals(vertices, faces, degen_norm)
    
    # Get exploded normals
    n = np.zeros(v.shape)
    for idx, face in enumerate(f):
        for x in range(3):
            n[face[x]] = normals[idx]

    # Plot
    plotShader(v, f, n)
    
# Flat Shading    
flat_shading(bunny_v, bunny_f)
flat_shading(cube_v, cube_f)
flat_shading(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-vertex Shading

In [7]:
def per_vertex_shading(vertices, faces) -> None:
    n = igl.per_vertex_normals(vertices, faces, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)
    plotShader(vertices, faces, n)
    

# Per Vertex Shading
per_vertex_shading(bunny_v, bunny_f)
per_vertex_shading(cube_v, cube_f)
per_vertex_shading(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-corner Shading

In [8]:
def per_corner_shading(vertices, faces) -> None:
    
    v, f = explode(vertices, faces)
    normals = igl.per_corner_normals(v, f, 50)
   
    # Get exploded normals
    # n = np.zeros(v.shape)
    # for idx, face in enumerate(f):
    #     for x in range(3):
    #         n[face[x]] = normals[idx]
    
    # Apparently doesn't need normal mapping
    plotShader(v, f, normals)
    
    
# Per Corner Shading
per_corner_shading(bunny_v, bunny_f)
per_corner_shading(cube_v, cube_f)
per_corner_shading(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Connected Components

In [9]:
def connected_components(vertices, faces) -> None:
    # Get components 
    components = igl.face_components(faces)
    meshplot.plot(vertices, faces, c = components)
    print(f'Number of connected components: {len(np.unique(components))}')
    
    size = np.zeros(len(np.unique(components)), dtype=int)
    for idx, component in enumerate(components):
         size[component] += 1
    
    for idx, s in enumerate(size):
        print(f'Size of component {idx} is {s}.')
    
# Connected Components
connected_components(bumpy_v, bumpy_f)
connected_components(bunny_v, bunny_f)
connected_components(car_v, car_f)
connected_components(cup_v, cup_f)
connected_components(cube_v, cube_f)
connected_components(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Number of connected components: 1
Size of component 0 is 2496.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Number of connected components: 1
Size of component 0 is 6966.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.194198…

...

Number of connected components: 2
Size of component 0 is 3360.
Size of component 1 is 2304.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Number of connected components: 1
Size of component 0 is 12.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Number of connected components: 1
Size of component 0 is 760.


## A simple subdivision scheme

In [11]:
def subdivision_scheme(vertices, faces) -> None:
    # Variables
    barycenters = igl.barycenter(vertices, faces)
    faces_double_prime = list()
    faces_prime = list()
    num_vertices = len(vertices)
    vertex_to_vertex = igl.adjacency_list(faces)
    new_positions = np.zeros(vertices.shape)
    triangle_triangle_list, _ = igl.triangle_triangle_adjacency(faces)
    
    
    # Create new vertices in center and conecting faces 
    for i, face in enumerate(faces):
        for j in range(3):
            faces_double_prime.append(
                [face[j], face[(j + 1) % 3], num_vertices + i]
            )
    
    # Optimize to be an np array for faster calculations
    faces_double_prime = np.array(faces_double_prime)

    # Move each old vertex to new position
    for i in range(len(new_positions)):
        valance = len(vertex_to_vertex[i])
        
        # Calculate new position
        sum = np.zeros(3)
        for j in range(valance):
            sum += vertices[vertex_to_vertex[i][j]]
            
        # Calculate a_n
        a_n = (4 - 2 * np.cos(2 * np.pi / valance)) / 9
        
        new_positions[i] = (1 - a_n) * vertices[i] + a_n / valance * sum
        
    vertices_prime = np.concatenate((new_positions, barycenters))

    # Flip edges
    for i, face in enumerate(faces):
        for j in range(3):
            faces_prime.append(
                [face[j], len(vertices) + triangle_triangle_list[i][j] , len(vertices) + i]
            )
            
    # Optimize Array and return
    faces_prime = np.array(faces_prime)
    
    # Plot
    meshplot.plot(vertices_prime, faces_prime, shading={'wireframe':True})
    
    return vertices_prime, faces_prime  
    

# Subdivision Scheme
subdivision_scheme(bumpy_v, bumpy_f)
subdivision_scheme(bunny_v, bunny_f)
subdivision_scheme(car_v, car_f)
subdivision_scheme(cup_v, cup_f)
subdivision_scheme(cube_v, cube_f)
subdivision_scheme(sphere_v, sphere_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016826…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.220233…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.253652…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.9802322…

(array([[ 0.11161591, -0.68817545, -0.03098417],
        [ 0.09657845, -0.68817545, -0.06395871],
        [ 0.07208716, -0.68817545, -0.09067279],
        ...,
        [ 0.06424533,  0.69425467,  0.03273467],
        [ 0.07121633,  0.69425467,  0.01127967],
        [ 0.07121633,  0.69425467, -0.01127967]]),
 array([[   0, 1102,  382],
        [   1,  383,  382],
        [  20,  421,  382],
        ...,
        [ 379, 1101, 1141],
        [ 360, 1122, 1141],
        [ 381, 1140, 1141]]))